# vectorized boolean operations

In [1]:
import Numeric.LinearAlgebra
:ext FlexibleContexts

## pretty printing

In [2]:
import IHaskell.Display
:ext FlexibleInstances

dec = 3

dispBool = ("\n"++) . format " " f
  where
    f 1 = "\\top"
    f 0 = "\\cdot"

instance IHaskellDisplay (Matrix I) where
   display m = return $ Display [html ("<p>$$"++(latexFormat "bmatrix" . dispBool) m++"$$</p>")]

instance IHaskellDisplay (Matrix C) where
   display m = return $ Display [html ("<p>$$"++(latexFormat "bmatrix" . dispcf dec) m++"$$</p>")]

instance IHaskellDisplay (Matrix R) where
   display m = return $ Display [html ("<p>$$"++ (latexFormat "bmatrix" . dispf dec) m++"$$</p>")]

## definitions

vectorized operators defined in terms of `step` and `cond`

In [3]:
-- specialized for Int result
cond'
    :: (Element t, Ord t, Container c I, Container c t)
    => c t -> c t -> c I -> c I -> c I -> c I
cond' = cond

In [4]:
infix  4  .==., ./=., .<., .<=., .>=., .>.
infixr 3  .&&.
infixr 2  .||.

a .<.  b = cond' a b 1 0 0
a .<=. b = cond' a b 1 1 0
a .==. b = cond' a b 0 1 0
a ./=. b = cond' a b 1 0 1
a .>=. b = cond' a b 0 1 1
a .>.  b = cond' a b 0 0 1

a .&&. b  = step (a*b)
a .||. b  = step (a+b)
no a      = 1-a
xor a b   = a ./=. b
equiv a b = a .==. b
imp a b   = no a .||. b

other useful functions

In [5]:
taut x = minElement x == 1

minEvery a b = cond a b a a b
maxEvery a b = cond a b b b a

eye n = ident n :: Matrix R

clip a b x = cond y b y y b
  where
    y = cond x a a x x

## examples

In [6]:
col [1..7] .<=. row [1..5]

$$\begin{bmatrix}
\top & \top & \top & \top & \top
\\
\cdot & \top & \top & \top & \top
\\
\cdot & \cdot & \top & \top & \top
\\
\cdot & \cdot & \cdot & \top & \top
\\
\cdot & \cdot & \cdot & \cdot & \top
\\
\cdot & \cdot & \cdot & \cdot & \cdot
\\
\cdot & \cdot & \cdot & \cdot & \cdot
\end{bmatrix}$$

In [7]:
m = (3><4) [1..] :: Matrix R

In [8]:
m

$$\begin{bmatrix}
1 & 2 & 3 & 4
\\
5 & 6 & 7 & 8
\\
9 & 10 & 11 & 12
\end{bmatrix}$$

In [9]:
clip 3 8 m

$$\begin{bmatrix}
3 & 3 & 3 & 4
\\
5 & 6 & 7 & 8
\\
8 & 8 & 8 & 8
\end{bmatrix}$$

In [10]:
find (>6) m

[(1,2),(1,3),(2,0),(2,1),(2,2),(2,3)]

In [11]:
(m .>=. 10)  .||.  (m .<. 4)

$$\begin{bmatrix}
\top & \top & \top & \cdot
\\
\cdot & \cdot & \cdot & \cdot
\\
\cdot & \top & \top & \top
\end{bmatrix}$$

In [12]:
cond (col [1..3]) (row [1..4]) m 50 (3*m)

$$\begin{bmatrix}
50 & 2 & 3 & 4
\\
15 & 50 & 7 & 8
\\
27 & 30 & 50 & 12
\end{bmatrix}$$

In [13]:
assoc (6,8) 7 $ zip (find (/=0) (eye 5)) [10..] :: Matrix Z

(6><8)
 [ 10,  7,  7,  7,  7, 7, 7, 7
 ,  7, 11,  7,  7,  7, 7, 7, 7
 ,  7,  7, 12,  7,  7, 7, 7, 7
 ,  7,  7,  7, 13,  7, 7, 7, 7
 ,  7,  7,  7,  7, 14, 7, 7, 7
 ,  7,  7,  7,  7,  7, 7, 7, 7 ]

In [14]:
accum (eye 5) (+) [((0,2),3), ((3,1),7), ((1,1),1)]

$$\begin{bmatrix}
1 & 0 & 3 & 0 & 0
\\
0 & 2 & 0 & 0 & 0
\\
0 & 0 & 1 & 0 & 0
\\
0 & 7 & 0 & 1 & 0
\\
0 & 0 & 0 & 0 & 1
\end{bmatrix}$$

In [15]:
p = fromList [0,0,1,1] :: Vector I
q = fromList [0,1,0,1] :: Vector I

In [16]:
fromColumns [p, q, p.&&.q, p .||.q, p `xor` q, p `equiv` q, p `imp` q]

$$\begin{bmatrix}
\cdot & \cdot & \cdot & \cdot & \cdot & \top & \top
\\
\cdot & \top & \cdot & \top & \top & \cdot & \top
\\
\top & \cdot & \cdot & \top & \top & \cdot & \cdot
\\
\top & \top & \top & \top & \cdot & \top & \top
\end{bmatrix}$$

In [17]:
taut $ (p `imp` q ) `equiv` (no q `imp` no p)

True

In [18]:
taut $ xor p q `equiv` (p .&&. no q .||. no p .&&. q)

False